# Davide Marchi
## Assignment 4 - Bayesian Network (BN)
_Implement a Bayesian Network (BN) comprising at least 10 nodes, all with binomial or multinomial distribution. Represent the BN with the data structures that you deem appropriate and in the programming language that you prefer. The BN should model some problem/process of your choice, so you are also free to define the topology according to your prior knowledge (just be ready to justify your choices). [...]<br>
Once you have modelled the BN, also plug in the necessary local conditional probability tables. You can set the values of the probabilities following your own intuition on the problem (ie no need to learn them from data). Then run some episoded of Ancestral Sampling on the BN and discuss the results.[...]_

### Imported modules
The realizations of the Bayesian Network starts by importing two needed modules: `random` and `itertools`:<br>
The first one will be needed to randomly generate the samples during ancestral sampling with the function `random()` while `itertools` will be usuful to create a list of all the possible permutations of the states of the parents of a node with `product()`.

In [1]:
# Import librarires
import random
import itertools

### Bayesian Network class
I have then implemented the `BayesianNetwork` class, which represent a Directed Acyclic Graph (DAG) whose nodes are random variables and the edges describe the conditional independence relationships.<br>
The nodes in the network are stored in a dictionary (`self.nodes`) where the keys are the names of the nodes and the values are dictionaries that contains the node's states, parents, and CPT. This structure allows for efficient lookup and modification of the nodes (since it's based on an hash map).<br>

This value dictionary contains the following keys:
- `states`: This key corresponds to a list of the possible states that the node can take.
- `parents`: This key corresponds to a list of the node's parent nodes.
- `cpt`: This key corresponds to the node's Conditional Probability Table (CPT).

It's important to note that the cpt is a dictionary aswell:<br>
- The keys of the CPT dictionary are tuples representing the states of the parent nodes. If a node has no parents, the key will be an empty tuple `()`.
- The values of the CPT dictionary are lists representing the probabilities of the node's states given these parent states. The order of the probabilities in the list corresponds to the order of the states in the node's states list.

In [2]:
# Define the Bayesian Network class
class BayesianNetwork:
    def __init__(self):
        self.nodes = {}
    
    # Add a node to the network after checking for validity
    def add_node(self, name, states, cpt, parents=[]):

        # Check if the node already exists in the network
        if name in self.nodes:
            print(f"Node '{name}' already exists in the network.")
            return
        
        # Check if the node has at least one state
        if len(states) == 0:
            print(f"Node '{name}' must have at least one state.")
            return
        
        # Check if all parent nodes exist in the network
        for parent in parents:
            if parent not in self.nodes:
                print(f"Parent node '{parent}' does not exist in the network.")
                return

        # Check if the CPT is valid
        for parent_combination in itertools.product(*[self.nodes[parent]['states'] for parent in parents]):

            # Check if all the possible permutations of the states of the parent nodes are in the CPT
            # Check if each entry in the cpt as the corrent number of probabilities and the sum of the probabilities is 1
            if parent_combination not in cpt or len(cpt[parent_combination]) != len(states) or sum(cpt[parent_combination]) != 1:
                print(f"Invalid CPT for node '{name}'.")
                return

        # Add the node to the network
        self.nodes[name] = {'states': states, 'parents': parents, 'cpt': cpt}
    
    # Calculate the probability of a state of a node given its parents' states
    def probability(self, node, parent_states, state):
        states = self.nodes[node]['states']
        return self.nodes[node]['cpt'][parent_states][states.index(state)]
    
    # Sample a state for a node given its parents' states
    def sample(self, node, parent_states):

        # Generating a random value and collecting the possible states of the node
        p = random.random()
        cumulative_prob = 0
        states = self.nodes[node]['states']

        # Loop through the states of the node and calculate the probability of each state
        for state in states:
            probability = self.probability(node, parent_states, state)
            cumulative_prob += probability

            # Return the state and its probability if the random value is less than the cumulative probability (= if the state is selected)
            if p <= cumulative_prob:
                return state, probability
    
    # Sample states for all nodes
    def sample_states(self):
        sampled_states = {}
        joint_probability = 1

        # Loop through all nodes and sample the state of each node
        for node in self.nodes:
            parents = self.nodes[node]['parents']
            parent_states = tuple(sampled_states[parent] for parent in parents)
            sampled_states[node], probaility = self.sample(node, parent_states)
            joint_probability *= probaility

        # Return the sampled states and the joint probability
        return sampled_states, joint_probability

### Network example
To show the functionality of the presented implementation here is presented an example of istantiation of an object of the `BayesianNetwork` class.

In [3]:
# Define the Bayesian Network for concert enjoyment
concert_bn = BayesianNetwork()

# Add nodes to the network with their conditional probability distributions (CPDs)

# Node: MusicGenre
concert_bn.add_node('MusicGenre', ['Pop', 'Rock', 'Jazz'], 
                    {
                        (): [0.45, 0.30, 0.25]
                    })

# Node: ArtitstFame
concert_bn.add_node('ArtistFame', ['Unknown', 'Known', 'Superstar'], 
                    {
                        (): [0.2, 0.7, 0.1]
                    })

# Node: ArtistPerformance
concert_bn.add_node('ArtistPerformance', ['Poor', 'Average', 'Excellent'], 
                    {
                        ('Pop', 'Unknown'): [0.1, 0.6, 0.3],
                        ('Pop', 'Known'): [0.2, 0.6, 0.2],
                        ('Pop', 'Superstar'): [0.3, 0.5, 0.2],
                        ('Rock', 'Unknown'): [0.2, 0.6, 0.2],
                        ('Rock', 'Known'): [0.3, 0.5, 0.2],
                        ('Rock', 'Superstar'): [0.4, 0.4, 0.2],
                        ('Jazz', 'Unknown'): [0.3, 0.5, 0.2],
                        ('Jazz', 'Known'): [0.4, 0.4, 0.2],
                        ('Jazz', 'Superstar'): [0.5, 0.3, 0.2]
                    },
                    parents=['MusicGenre', 'ArtistFame'])

# Node: VenueAtmosphere
concert_bn.add_node('VenueAtmosphere', ['Dull', 'Energetic', 'Electrifying'], 
                    {
                        ('Rock',): [0.1, 0.6, 0.3],
                        ('Jazz',): [0.4, 0.2, 0.4],
                        ('Pop',): [0.1, 0.3, 0.6]
                    },
                    parents=['MusicGenre'])

# Node: CrowdExcitement
concert_bn.add_node('CrowdExcitement', ['Low', 'Moderate', 'High'], 
                    {
                        ('Dull', 'Poor'): [0.1, 0.6, 0.3],
                        ('Dull', 'Average'): [0.2, 0.6, 0.2],
                        ('Dull', 'Excellent'): [0.3, 0.5, 0.2],
                        ('Energetic', 'Poor'): [0.2, 0.6, 0.2],
                        ('Energetic', 'Average'): [0.3, 0.5, 0.2],
                        ('Energetic', 'Excellent'): [0.4, 0.4, 0.2],
                        ('Electrifying', 'Poor'): [0.3, 0.5, 0.2],
                        ('Electrifying', 'Average'): [0.4, 0.4, 0.2],
                        ('Electrifying', 'Excellent'): [0.5, 0.3, 0.2]
                    },
                    parents=['VenueAtmosphere', 'ArtistPerformance'])

# Node: TicketPrice
concert_bn.add_node('TicketPrice', ['Cheap', 'Moderate', 'Expensive'], 
                    {
                        ('Unknown',): [0.8, 0.1, 0.1],
                        ('Known',): [0.2, 0.6, 0.2],
                        ('Superstar',): [0.1, 0.2, 0.7]
                    },
                    parents=['ArtistFame'])

# Node: WeatherCondition
concert_bn.add_node('WeatherCondition', ['Sunny', 'Cloudy', 'Rainy'], 
                    {
                        (): [0.4, 0.4, 0.2]
                    })

# Node: TravelTime
concert_bn.add_node('TravelTime', ['Moderate', 'Long'], 
                    {
                        ('Sunny',): [0.9, 0.1],
                        ('Cloudy',): [0.7, 0.3],
                        ('Rainy',): [0.2, 0.8]
                    },
                    parents=['WeatherCondition'])

# Node: ConcertExpectations
concert_bn.add_node('ConcertExpectations', ['Low', 'Moderate', 'High'], 
                    {
                        (): [0.3, 0.4, 0.3]
                    })

# Node: OverallEnjoyment (Target Variable)
concert_bn.add_node('OverallEnjoyment', ['Low', 'Moderate', 'High'], 
                    {
                        ('Moderate', 'Low', 'Low'): [0.1, 0.7, 0.2],
                        ('Moderate', 'Low', 'Moderate'): [0.05, 0.9, 0.05],
                        ('Moderate', 'Low', 'High'): [0.01, 0.98, 0.01],
                        ('Moderate', 'Moderate', 'Low'): [0.05, 0.8, 0.15],
                        ('Moderate', 'Moderate', 'Moderate'): [0.01, 0.95, 0.04],
                        ('Moderate', 'Moderate', 'High'): [0.001, 0.999, 0],
                        ('Moderate', 'High', 'Low'): [0.02, 0.7, 0.28],
                        ('Moderate', 'High', 'Moderate'): [0.001, 0.99, 0.009],
                        ('Moderate', 'High', 'High'): [0, 0.9, 0.1],
                        ('Long', 'Low', 'Low'): [0.05, 0.8, 0.15],
                        ('Long', 'Low', 'Moderate'): [0.01, 0.95, 0.04],
                        ('Long', 'Low', 'High'): [0.001, 0.999, 0],
                        ('Long', 'Moderate', 'Low'): [0.02, 0.7, 0.28],
                        ('Long', 'Moderate', 'Moderate'): [0.001, 0.99, 0.009],
                        ('Long', 'Moderate', 'High'): [0, 0.9, 0.1],
                        ('Long', 'High', 'Low'): [0, 0.8, 0.2],
                        ('Long', 'High', 'Moderate'): [0, 0.95, 0.05],
                        ('Long', 'High', 'High'): [0, 0.99, 0.01]
                    },
                    parents=['TravelTime', 'CrowdExcitement', 'ConcertExpectations'])

# Print the nodes in the network
print(concert_bn.nodes)

{'MusicGenre': {'states': ['Pop', 'Rock', 'Jazz'], 'parents': [], 'cpt': {(): [0.45, 0.3, 0.25]}}, 'ArtistFame': {'states': ['Unknown', 'Known', 'Superstar'], 'parents': [], 'cpt': {(): [0.2, 0.7, 0.1]}}, 'ArtistPerformance': {'states': ['Poor', 'Average', 'Excellent'], 'parents': ['MusicGenre', 'ArtistFame'], 'cpt': {('Pop', 'Unknown'): [0.1, 0.6, 0.3], ('Pop', 'Known'): [0.2, 0.6, 0.2], ('Pop', 'Superstar'): [0.3, 0.5, 0.2], ('Rock', 'Unknown'): [0.2, 0.6, 0.2], ('Rock', 'Known'): [0.3, 0.5, 0.2], ('Rock', 'Superstar'): [0.4, 0.4, 0.2], ('Jazz', 'Unknown'): [0.3, 0.5, 0.2], ('Jazz', 'Known'): [0.4, 0.4, 0.2], ('Jazz', 'Superstar'): [0.5, 0.3, 0.2]}}, 'VenueAtmosphere': {'states': ['Dull', 'Energetic', 'Electrifying'], 'parents': ['MusicGenre'], 'cpt': {('Rock',): [0.1, 0.6, 0.3], ('Jazz',): [0.4, 0.2, 0.4], ('Pop',): [0.1, 0.3, 0.6]}}, 'CrowdExcitement': {'states': ['Low', 'Moderate', 'High'], 'parents': ['VenueAtmosphere', 'ArtistPerformance'], 'cpt': {('Dull', 'Poor'): [0.1, 0.6, 0

### Ancestral Sampling
To assess the ability to perform the Ancestral Sampling i went for multiple calls to the `sample_states()` function.
A certain number of iterations will give us enough data to try to make assumptions regarding which states are most likely to be reached ny the nodes.

In [4]:
# Initializing a list that will contain the overall enjoyment of each episode
OverallEnjoymentList = []

# Cycle to perform multiple episodes of sampling
for i in range(10000):

    # Sample states for all nodes
    sampled_states, joint_probability = concert_bn.sample_states()

    # Print sampled states
    for node, state in sampled_states.items():
        print(f"{node}: {state}")

    # Print joint probability
    print(f"Joint Probability: {joint_probability}")

    # Append the overall enjoyment to the list
    OverallEnjoymentList.append(sampled_states['OverallEnjoyment'])

# Print the percentage of occurrence of each level of overall enjoyment
print(f"Low: {OverallEnjoymentList.count('Low')/len(OverallEnjoymentList)*100}%")
print(f"Moderate: {OverallEnjoymentList.count('Moderate')/len(OverallEnjoymentList)*100}%")
print(f"High: {OverallEnjoymentList.count('High')/len(OverallEnjoymentList)*100}%")

MusicGenre: Pop
ArtistFame: Known
ArtistPerformance: Average
VenueAtmosphere: Electrifying
CrowdExcitement: High
TicketPrice: Moderate
WeatherCondition: Cloudy
TravelTime: Moderate
ConcertExpectations: High
OverallEnjoyment: High
Joint Probability: 0.0001143072
MusicGenre: Jazz
ArtistFame: Known
ArtistPerformance: Poor
VenueAtmosphere: Energetic
CrowdExcitement: Moderate
TicketPrice: Moderate
WeatherCondition: Cloudy
TravelTime: Moderate
ConcertExpectations: Moderate
OverallEnjoyment: Moderate
Joint Probability: 0.0005362559999999999
MusicGenre: Jazz
ArtistFame: Superstar
ArtistPerformance: Average
VenueAtmosphere: Energetic
CrowdExcitement: Moderate
TicketPrice: Moderate
WeatherCondition: Cloudy
TravelTime: Moderate
ConcertExpectations: High
OverallEnjoyment: Moderate
Joint Probability: 1.25874e-05
MusicGenre: Pop
ArtistFame: Unknown
ArtistPerformance: Average
VenueAtmosphere: Electrifying
CrowdExcitement: Moderate
TicketPrice: Moderate
WeatherCondition: Sunny
TravelTime: Moderate
Con

### Final considerations
